# Module 2 Summative Lab

## Introduction

For today's section, we're going to work on a single big lab to apply everything we've learned in Module 2!

## About This Lab

A quick note before getting started--this lab isn't like other labs you seen so far. This lab is meant to take ~8 hours to complete, so it's much longer and more challenging that the average labs you've seen so far. If you feel like this lab is challenging or that you might be struggling a bit, don't fret--that's by design! With everything we've learned about Web Scraping, APIs, and Databases, the best way to test our knowledge of it is to build something substantial! 

## The Project

In this lab, we're going to make use of everything we've learned about APIs, databases, and Object-Oriented Programming to **_Extract, Transform, and Load_** (or **_ETL_**, for short) some data from a SQL database into a MongoDB Database. 

You'll find a database containing information about soccer teams and the matches they've played in the file `database.sqlite`. For this project, our goal is to get the data we think is important from this SQL database, do some calculations and data transformation, and then store everything in a MongoDB database. 

Let's get into the specifics of this project.

### The Goal

Start by examining the data dictionary for the SQL database we'll be working with, which comes from this [kaggle page](https://www.kaggle.com/laudanum/footballdelphi).  Familiarize yourself with the tables it contains, and what each column means. We'll be using this database to get data on each soccer team, calculate some summary statistics, and then store each in a MongoDB database. 

Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season. 

#### Getting the Weather Data

Note that for this last calculation, you'll need to figure out if it was raining or not during the game. The database itself does not contain this information, but it does contain the date on which the game was played. For this, you'll need to use the [DarkSky API](https://darksky.net/dev) to get the historical weather data for that day. Note that each game is played in a different location, and this information is not contained in our SQL database. However, the teams in this database are largely german, so go ahead and just use the weather in Berlin, Germany as a proxy for this information. If it was raining in Berlin on the day the game was played, count that as rain game--**_you do not need to try and figure out the actual weather at each game's location, because we don't have that information!_**

#### NOTE: The DarkSky API is limited to 1000 free API calls a day, so be sure to test your model on very small samples. Otherwise, you'll hit the rate limit!

## Project Architecture

Unlike previous labs, this lab is more open-ended, and will require you to make design decisions and plan out your strategy for building a system with this many working parts. However, **_using Object-Oriented Programming is a requirement for this project--you must create at least 2 separate, well structured classes in your solution!_** Although it may seem easier to "just start coding", this is a classic beginner's mistake. Instead, think about separating out the different functionalities you'll need to reach your goal, and then build classes to handle each. For instance, at minimum, you'll need to:

* Query the SQL database
* Calculate summary statistics
* Get the weather data from the DarkSky API
* Load the data into MongoDB

We **_strongly recommend_** you consider creating separate classes for handling at least some of these of these tasks.  Be sure to plan the inputs, outputs, and methods for each class before you begin coding! 

**_NOTE:_** We have provided some empty classes below. You are welcome to delete them and use a different architecture for this project if you so choose.  You do not have to use each of them, they are just there to give you an idea of what you could sorts of classes you may want to consider using.

### Rapid Prototyping and Refactoring

It's totally okay to try to get a task working without using OOP. For instance, when experimenting with the DarkSky API for getting historical weather data, it makes sense to just write the code in the cells and rapidly iterate until you get it all working. However, once you get it working, you're not done--you should then **_Refactor_** your code into functions or classes to make your code more modular, reusable, understandable, and maintainable! 

In short--do what you need to do to get each separate piece of functionality working, and then refactor it into a class after you've figured it out!

### Some Final Advice

You haven't built anything this big or complex thus far, so you may not yet fully realize how much trial and error goes into it. If your code keeps breaking, resist the urge to get frustrated, and just keep working. Software development is an iterative process!  No one writes perfect code that works the first time for something this involved. You're going to run into _a lot_ of small errors in this project, right up until the point where it just works, and then you're done! However, you can reduce these errors by planning out your code, and thinking about how all of the pieces fit together before you begin coding. Once you have some basic understanding of how it all will work, then you'll know what you need to build, and then all that is left is to build it!

In short:

* Plan ahead--you'll thank yourself later!
* Errors and broken code aren't bad, they're normal. 
* Keep working, and stay confident--you can do this!

Good luck--we look forward to seeing your completed project!

In [ ]:
# You don't have to use these classes, but we recommend them as a good place to start!
class WeatherGetter():
    pass

In [ ]:
class MongoHandler():
    pass

# Summary

In this lab, we dug deep and used everything we've learned so far about python programming, databases, HTTP requests and API calls to ETL data from a SQL database into a MongoDB instance!

In [2]:
import requests
import sqlite3
import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import datetime




In [19]:
# connect to sqlite database and set up cursor
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [9]:
# head of sqlite_sequence table
c.execute("""select * from sqlite_sequence limit 5;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,name,seq
0,Matches,46774
1,Unique_Teams,128


In [5]:
# head of Matches table
c.execute("""select * from Matches limit 5;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


In [7]:
# head of Teams_in_Matches table
c.execute("""select * from Teams_in_Matches limit 5;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


In [20]:
# head of Unique_Teams table
c.execute("""select * from Unique_Teams;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
all_teams = df['TeamName'].unique()
print(all_teams)
df.head()

['Bayern Munich' 'Dortmund' 'Leverkusen' 'RB Leipzig' 'Schalke 04'
 "M'gladbach" 'Wolfsburg' 'FC Koln' 'Hoffenheim' 'Hertha' 'Mainz'
 'Hamburg' 'Werder Bremen' 'Ein Frankfurt' 'Augsburg' 'Freiburg'
 'Stuttgart' 'Hannover' 'Ingolstadt' 'Darmstadt' 'Paderborn' 'Nurnberg'
 'Braunschweig' 'Greuther Furth' 'Fortuna Dusseldorf' 'Kaiserslautern'
 'St Pauli' 'Bochum' 'Karlsruhe' 'Bielefeld' 'Cottbus' 'Duisburg'
 'Hansa Rostock' 'Aachen' 'Union Berlin' 'Heidenheim' 'Dresden'
 'Sandhausen' 'Erzgebirge Aue' 'Kiel' 'Regensburg' 'Munich 1860'
 'Wurzburger Kickers' 'Frankfurt FSV' 'Aalen' 'Oberhausen' 'Osnabruck'
 'Koblenz' 'Ahlen' 'Wehen' 'CZ Jena' 'Offenbach' 'Essen' 'Burghausen'
 'Unterhaching' 'Saarbrucken' 'Siegen' 'Babelsberg' 'Chemnitz' 'Ein Trier'
 'Erfurt' 'F Koln' 'Gutersloh' 'Homburg' 'Leipzig' 'Lubeck' 'Mannheim'
 'Meppen' 'Oldenburg' 'Reutlingen' 'Schweinfurt' 'Stuttgarter K'
 'TB Berlin' 'Uerdingen' 'Ulm' 'Wattenscheid' 'Wuppertaler' 'Zwickau'
 'RW Essen' "M'Gladbach" 'Arsenal' 'Aston 

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [23]:
# head of Teams table
c.execute("""select * from Teams where season = '2011';""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
all_teams = df['TeamName'].unique()
print(all_teams)
print(len(all_teams))
df.head()

['Bayern Munich' 'Dortmund' 'Leverkusen' 'Schalke 04' 'Werder Bremen'
 'Wolfsburg' 'Hamburg' 'Stuttgart' 'Hoffenheim' 'FC Koln' 'Hannover'
 "M'gladbach" 'Freiburg' 'Mainz' 'Hertha' 'Kaiserslautern' 'Nurnberg'
 'Augsburg' 'Ein Frankfurt' 'St Pauli' 'Cottbus' 'Bochum'
 'Fortuna Dusseldorf' 'Greuther Furth' 'Munich 1860' 'Aachen' 'Ingolstadt'
 'Duisburg' 'Karlsruhe' 'Dresden' 'Frankfurt FSV' 'Erzgebirge Aue'
 'Hansa Rostock' 'Union Berlin' 'Paderborn' 'Braunschweig']
36


,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2011,Bayern Munich,24,25,11,335600000,13980000,75000
1,2011,Dortmund,28,24,12,158200000,5650000,81359
2,2011,Leverkusen,29,23,9,151100000,5210000,30210
3,2011,Schalke 04,37,24,20,136730000,3700000,62271
4,2011,Werder Bremen,38,23,17,125350000,3300000,42100


In [20]:
# Master query

# c.execute("""SELECT Match_ID, Unique_Team_ID, Unique_Team, Season, Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR 
#             FROM FlatView_Advanced WHERE Season = '2011' GROUP BY Match_ID ORDER BY Match_ID;""")

c.execute("""select * from FlatView_Advanced where season = '2011' limit 2;""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(df.columns)
df

Index(['Match_ID', 'Unique_Team_ID', 'Unique_Team', 'Div', 'Season', 'Date',
       'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'KaderHome',
       'AvgAgeHome', 'ForeignPlayersHome', 'OverallMarketValueHome',
       'AvgMarketValueHome', 'StadiumCapacity'],
      dtype='object')


,Match_ID,Unique_Team_ID,Unique_Team,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,1491,1,Bayern Munich,D1,2011,2011-08-07,Bayern Munich,M'gladbach,0,1,A,24,25,11,335600000,13980000,75000
1,1094,1,Bayern Munich,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,24,25,11,335600000,13980000,75000


In [72]:
# get all unique teams
teams = df.HomeTeam.append(df.AwayTeam)
unique_teams = set(teams)
list(unique_teams)

{'Augsburg',
 'Bayern Munich',
 'Dortmund',
 'FC Koln',
 'Freiburg',
 'Hamburg',
 'Hannover',
 'Hertha',
 'Hoffenheim',
 'Kaiserslautern',
 'Leverkusen',
 "M'gladbach",
 'Mainz',
 'Nurnberg',
 'Schalke 04',
 'Stuttgart',
 'Werder Bremen',
 'Wolfsburg'}

In [ ]:
# average statistics methods

def __init__(self):
    pass

def total_goals(team):
    goals = []
    for unique_team in df.Unique_Team:
        for n in range(len(df.Unique_Team)):
            if team == df.HomeTeam[n]:
                goals.append(df.FTHG[n])
            elif team == df.AwayTeam[n]:
                goals.append(df.FTAG[n])
        # print(goals)
        return sum(goals)

def total_wins(team):
    wins = []
    for unique_team in df.Unique_Team:
        for n in range(len(df.Unique_Team)):
            if team == df.HomeTeam[n] and df.FTR[n] == 'H':
                wins.append(1)
            elif team == df.AwayTeam[n] and df.FTR[n] == 'A':
                wins.append(1)
        return sum(wins)

def total_losses(team):
    loss = []
    for unique_team in df.Unique_Team:
        for n in range(len(df.Unique_Team)):
            if team == df.HomeTeam[n] and df.FTR[n] == 'A':
                loss.append(1)
            elif team == df.AwayTeam[n] and df.FTR[n] == 'H':
                loss.append(1)
        return sum(loss)

def get_teams(df):
    teams = df.HomeTeam.append(df.AwayTeam)
    unique_teams = list(set(teams))
    return unique_teams

        
print('Total Goals:', WeatherGetter.total_goals('Stuttgart'))
print('Total Wins:', WeatherGetter.total_wins('Stuttgart'))
print('Total Losses:', WeatherGetter.total_losses('Stuttgart'))


In [ ]:
# plot wins vs losses histogram
def plot_hist(team):
    wins_losses = [total_wins(team), total_loses(team)]
    sns.distplot(wins_losses, kde = False)

In [ ]:
# show all summary stats

def team_stats(df):
    for idx, team in enumerate(WeatherGetter.get_teams(df)):
        print([team, WeatherGetter.total_goals(team),WeatherGetter.total_wins(team),WeatherGetter.total_losses(team)])

team_stats(df)

In [17]:
# test DarkSky API 
date = '2011-08-07'
time = str(int(datetime.datetime.strptime(date, "%Y-%m-%d").timestamp()))
response = requests.get('https://api.darksky.net/forecast/13e9efb8811df5d1b55434bb80abe89e/52.520008,13.404954,' + time).json()
response['currently']['icon']


'partly-cloudy-day'

In [ ]:
# wins on rainy days / total wins

In [39]:
# 'rainy_day' feature engineering for every game day in 2011 using DarkSky.net weather API
df['Rainy_Day'] = False

for index, row in df.iterrows():
    time = str(int(datetime.datetime.strptime(row['Date'], "%Y-%m-%d").timestamp())) # unix timestamp from date, casting for API format
    response = requests.get('https://api.darksky.net/forecast/13e9efb8811df5d1b55434bb80abe89e/52.520008,13.404954,' + time).json()
    # print(response['currently']['icon'])
    # print(row['Rainy_Day'])
    if 'rain' in response['currently']['icon']:
        df.at[index,'Rainy_Day'] = True
    else:
        df.at[index,'Rainy_Day'] = False


    
df.head()
    
    
    
    
    

partly-cloudy-day
False
partly-cloudy-day
False


,Match_ID,Unique_Team_ID,Unique_Team,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity,Rainy_Day
0,1491,1,Bayern Munich,D1,2011,2011-08-07,Bayern Munich,M'gladbach,0,1,A,24,25,11,335600000,13980000,75000,True
1,1094,1,Bayern Munich,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,24,25,11,335600000,13980000,75000,True


In [ ]:
# calculate percentage rainy day wins

def rainy_wins(team):
    rainy_wins = []
    for unique_team in df.Unique_Team:
        for n in range(len(df.Unique_Team)):
            if team == df.HomeTeam[n] and df.FTR[n] == 'H' and df.Rainy_Day[n] == True:
                rainy_wins.append(1)
            elif team == df.AwayTeam[n] and df.FTR[n] == 'A' and df.Rainy_Day[n] == True:
                rainy_wins.append(1)
        return sum(rainy_wins)
    
def rainy_wins_perc(team):
    rainy_wins(team)/total_wins(team)

In [ ]:
# create DataFrame for MongoDB migration
# columns teamName, totalGoals, totalWins, wins_losses_viz, perc_rain_wins

def getYrStats(df):
    statsDict = {}
    for idx, team in get_teams(df):
        statsDict[idx] = {
            teamName = team, 
            totalGoals = total_goals(team), 
            totalWins = total_wins(team), 
            wins_losses_viz = wins_losses_viz(team), 
            perc_rain_wins = rainy_wins_perc(team)
        }
    return statsDict # we can feed this directly to MongoDB
        
statsDF = pd.DataFrame.from_dict(getYrStats(df))       


In [ ]:
# dump DataFrame (or dict) to MongoDB instance






In [ ]:
# Each record in the Mongo DB should hold the following:

# The name of the team
# The total number of goals scored by the team during the 2011 season
# The total number of wins the team earned during the 2011 season
# A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
# The team's win percentage on days where it was raining during games in the 2011 season.






